# p.20 Twitter Sentiment Analysis

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('SB8ckgT8l9c')

## 1. Twitter API v1.1

Please follow the video tutorial at https://pythonprogramming.net/twitter-api-streaming-tweets-python-tutorial/.

### 1.1 Preparation

(1) Get the twitter consumer key, consumer secret, access token, and access token secret.

Go to https://apps.twitter.com, sign in with your twitter credential, and create a new application if you don't have one. Then create your access token.

(2) Install the tweepy package.

tweepy is not the only available twitter API package, but it is the most applicable here.

Via pip:

```bash
$ pip install tweepy
```

or via conda:

```bash
$ conda install -c conda-forge tweepy
```

### 1.2 Stream tweets.

In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

# Consumer key
ckey = 'xxxxxxxx'
# Consumer secret
csecret = 'xxxxxxxx'
# Access token
atoken = 'xxxxxxxx'
# Access token secret
asecret = 'xxxxxxxx'

class listener(StreamListener):
    
    def on_data(self, data):
        print(data)
        return True
    
    def on_error(self, status):
        print(status)
        
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["car"])

### 1.3 Save and clean up tweets.

(1) Save tweets into a CSV file.

In [ ]:
import json
import time
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

# Consumer key
ckey = 'xxxxxxxx'
# Consumer secret
csecret = 'xxxxxxxx'
# Access token
atoken = 'xxxxxxxx'
# Access token secret
asecret = 'xxxxxxxx'

class listener(StreamListener):
    
    def on_data(self, data):
        try:
            #print(data)
            
            all_data = json.loads(data)
            tweet = all_data["text"]
            username = all_data["user"]["screen_name"]
            
            print(tweet)
            
            saveThis = str(time.time()) + '::' + username + '::' + tweet
            saveThis.encode('utf-8')
            
            saveFile = open('tweetDB2.csv', 'a')
            saveFile.write(saveThis)
            saveFile.write('\n')
            saveFile.close()
            return True
        except BaseException as e:
            print('failed on_data,', str(e))
            time.sleep(5)
    
    def on_error(self, status):
        print(status)
        
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["car"])

(2) Save tweets into a MySQL database.

In [ ]:
import json
import MySQLdb
import time
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

# Configuration for the local MySQL server:
# The default socket /tmp/mysql.sock doesn't exist and we haven't found a way of 
# setting it to the correct value, so we have to specify it in the connect() call. 
# Also, to work around the socket issue, we may specify the host as "127.0.0.1", 
# but it will force the client to connect to the MySQL server via TCP/IP, which is 
# in general slower than "localhost" via UNIX domain socket.
socketFile = '/var/run/mysqld/mysqld.sock' 
host = "localhost"
username = "root"
password = "xxxxxxxx"
dbname = "tutorial"

conn = MySQLdb.connect(
    unix_socket = socketFile,
    host = host, 
    user = username, 
    passwd = password, 
    db = dbname)
cur = conn.cursor()

# To work with foreign characters, use UTF-8 encoding.
# Also run the following commands in MySQL.
# mysql> SET NAMES utf8mb4;
# mysql> ALTER DATABASE tutorial CHARACTER SET = utf8mb4 COLLATE = utf8mb4_general_ci;
# Then verify the change in MySQL.
# mysql> SHOW VARIABLES WHERE Variable_name LIKE 'character\_set\_%' OR Variable_name LIKE 'collation%';
conn.set_character_set('utf8mb4')
cur.execute('SET NAMES utf8mb4;')
cur.execute('SET CHARACTER SET utf8mb4;')
#cur.execute('SET character_set_connection=utf8mb4;')

# Consumer key
ckey = 'xxxxxxxx'
# Consumer secret
csecret = 'xxxxxxxx'
# Access token
atoken = 'xxxxxxxx'
# Access token secret
asecret = 'xxxxxxxx'

class listener(StreamListener):
    
    def on_data(self, data):
        try:
            #print(data)
            
            all_data = json.loads(data)
            tweet = all_data["text"]
            username = all_data["user"]["screen_name"]
            
            cur.execute("INSERT INTO taula (time, username, tweet) VALUES (%s, %s, %s)",
                (time.time(), username, tweet))
            
            conn.commit()
            
            print((username, tweet))
            
            return True
        except BaseException as e:
            print('failed on_data,', str(e))
            time.sleep(5)
    
    def on_error(self, status):
        print(status)
        
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["car"])

### 2. Twitter sentiment analysis

In [1]:
import json
import time
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

# Import Twitter API credentials:
# Consumer key: ckey 
# Consumer secret: csecret
# Access token: atoken 
# Access token secret: asecret 
from twitterapi_cred import *

# Import our module of sentiment module
import sentiment_mod as s

class listener(StreamListener):
    
    def on_data(self, data):
        try:
            #print(data)
            
            all_data = json.loads(data)
            tweet = all_data["text"]
            
            sentiment_value, confidence = s.sentiment(tweet)
            
            print("tweet::", tweet)
            print("sentiment_value::", sentiment_value)
            print("confidence::", confidence)
            
            if confidence*100 >= 80:
                output = open("twitter-out.txt", "a")
                output.write(sentiment_value)
                output.write('\n')
                output.close()
            
            return True
        except BaseException as e:
            print('failed on_data,', str(e))
            time.sleep(5)
    
    def on_error(self, status):
        print(status)
        
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["trump"])

tweet:: RT @RawStory: Rachel #Maddow may have discovered the legal defense fund for Trump staffers caught up in the Russia probes https://t.co/mwxp…
sentiment_value:: pos
confidence:: 1.0
tweet:: RT @SabuesoDigital_: ► La razón por la que Trump tiene que silenciar a Rusia
 
https://t.co/BxsVxIiui0 https://t.co/ekQlqmRAc9
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @Imperator_Rex3: @Comey 'Hi, my name's Jim. As FBI Director, I cleared Hillary Clinton of her crimes under the Espionage Act. After that…
sentiment_value:: neg
confidence:: 0.8
tweet:: Online right now at https://t.co/RTr5dzsNOL
It's free to join! #freecams #freechat #sexcams #camgirl #camboy… https://t.co/mFMIPbIuhl
sentiment_value:: neg
confidence:: 1.0
tweet:: RT @krassenstein: Trump:  My State of the Union Had the Most Viewers Ever!

Apparently he's no good at simple arithmetic:

45.6 million  -…
sentiment_value:: pos
confidence:: 1.0
tweet:: RT @Gingrich_of_PA: DOJ/FBI to President Trump: “How dare you release our 

tweet:: RT @mmpadellan: All the hoopla over the memo reeks of desperation from the trump WH to change the subject.

Kinda like pouring kerosene on…
sentiment_value:: neg
confidence:: 1.0
tweet:: FULL: President Trump Speech at Republican Conference in West Virginia 2... https://t.co/xSwjkdBnFj
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @syeddoha: After #WI #State seat loss in #deepred district the entire #GOP establishment is throwing their weight on #PA house seat in a…
sentiment_value:: neg
confidence:: 1.0
tweet:: RT @TheAtlantic: Is Trump preparing for war with North Korea? @PeterBeinart writes that the North Korean portion of SOTU, and fate of Victo…
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @dee3598: The key players in Trump's #deepstate conspiracy theory are all #Republicans https://t.co/TvDF6r9GRO via @HuffPostPol
sentiment_value:: neg
confidence:: 1.0
tweet:: RT @JynErso_2017: Parents, siblings, and grown children are NOT "distant relatives," Mr. Trump.

And the

tweet:: RT @JCTheResistance: Reminder,  while dirty @DevinNunes waves the memo as a distraction 

👉 Pompeo said Russia WILL interfere in our 2018 e…
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @AbbyMartin: The corporate Democrats have no platform. Their slogan for 2018 is literally "I mean, have you seen the other guy?" When I…
sentiment_value:: neg
confidence:: 1.0
tweet:: RT @OliverMcGee: Obama didn't have an answer for jobs leaving America. He also falsely claimed @realdonaldtrump didn't have answer either.…
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @susisnyder: Because nuclear weapons are not helpful , put soldiers at greater risk of immediate incineration, as well as illegally thre…
sentiment_value:: pos
confidence:: 1.0
tweet:: RT @docrocktex26: How can *you* not talk about the reality that the intelligence community has repeatedly confirmed that *your* anti-Hillar…
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @JasonHalle: TRUMPS PARENTS WERE IMMIGRANTS

Then T

tweet:: @ThomasWictor Trump interviewed Mueller the DAY BEFORE Mueller was appointed Special Counsel.

Supposedly, it was a… https://t.co/aVb5XxExHl
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @thehill: Trump Justice Dept quietly shutters Obama-era office that expanded legal aid to poor citizens: report https://t.co/GLj0iTuskM…
sentiment_value:: pos
confidence:: 0.6
tweet:: @Talenteduser @ThreeWomen_ @KarenSh52112078 @Sessionbeer1 @brightsider123 @guyverhofstadt @Europarl_EN People will… https://t.co/PO1mKnNwzt
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @thehill: Trump after rejecting bipartisan fixes to DACA: "Blame the Dems" if no deal is reached https://t.co/onAtdr6Qp0 https://t.co/uS…
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @keithellison: This is horrifying. Trump's Dept. of Labor wants to enact a rule allowing restaurant bosses to control tips given to thei…
sentiment_value:: pos
confidence:: 1.0
tweet:: RT @ArthurSchwartz: Apparently Schiff and Flake use th

tweet:: @krassenstein Trump happened. #fakepres
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @ManhattanManOne: Resulta increíble que quienes con Trump son conocidos como  #Dreamers, Clinton los llamara directamente #Aliens. Pero…
sentiment_value:: pos
confidence:: 0.6
tweet:: RT @SafetyPinDaily: Trump is engaged in a slow motion Saturday Night Massacre   |Via Salon https://t.co/R5prGVOTnY
sentiment_value:: neg
confidence:: 1.0
tweet:: Let's hope that Trump can beat the UN out of him!!! https://t.co/Lop8yoz5Em
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @Kasparov63: Bingo. One of the reasons propaganda and lies work despite a free &amp; aggressive press. They still feel obliged to treat ever…
sentiment_value:: neg
confidence:: 1.0
tweet:: RT @Hoboscratch: @HouseDemocrats @realDonaldTrump I thought #trump and #Republicans said they stand with law enforcement turns out that was…
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @KarenChestney: Democrats reveal Devin Nunes is un

...

tweet:: @womensmarch https://t.co/8XHeBRCWEe
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @davidschneider: However nasty things get, at least Brexiters aren’t like Trump’s mob, accusing those who work for the government of dec…
sentiment_value:: neg
confidence:: 1.0
tweet:: RT @shwe_new: bb @tester2041994 trump
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @Independent: Morning Joe pulls abruptly ends interview with 'Fire and Fury' author over Trump affair allegations https://t.co/DSsmfE9VW…
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @davidfrum: And the GOP will be (even) more enabling of Trump in 2018 than 2017. "Release the memo" in plain English means "We will supp…
sentiment_value:: neg
confidence:: 1.0
tweet:: Where can you study abroad for free?
https://t.co/067PmNC3IV
#Trump #Pence
sentiment_value:: neg
confidence:: 1.0
tweet:: RT @krassenstein: Wow!! Comey basically just called Trump and Nunes "weasels and liars". https://t.co/rPzcmjNvUd
sentiment_value:: neg
co

tweet:: RT @kumailn: Today we found out:

Witness telling Mueller that Hope Hicks obstructed justice.
Trump asked Rosenstein if he was "on his side…
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @EdKrassen: BOOM!  Another misleading Trump stat has been debunked:

The AFL-CIO Chief says that 93,000 jobs were outsourced under Trump…
sentiment_value:: neg
confidence:: 1.0
tweet:: RT @a_fly_guy: WATCH: Trump mocks #TheResistance' at RNC winter meeting https://t.co/uT86ZEu53e
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @chargergreg3: #LetItDie and good riddance. https://t.co/JrfngD2Z4s
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @JoyAnnReid: President Obama: 48 million on 11 outlets  in 2010 
- George W. Bush: 52 million on 8 outlets in 2002
- Bill Clinton: 45.8…
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @TeamCoco: #Conan jokes about Trump, the #SuperBowl and more @ https://t.co/eoENMEmNsY https://t.co/y0dBsR6GbS
sentiment_value:: neg
confidence:: 0.8
tweet:: Thanks 

POTUS will lose thi…tee: BLUF: If the Nunes memo is released, it's all-out, scorched-earth war between the Trump WH &amp; FBI/IC. 
sentiment_value:: pos
confidence:: 0.8
tweet:: RT @Thomas1774Paine: ICYMI, THE TRUMP EFFECT: DNC Is Nearly Broke https://t.co/xs0Awl4mD7
sentiment_value:: neg
confidence:: 0.8
tweet:: #DictaduraNoCuentaVotos #YoSoyOscarPerez Lo que le pidió Trump al Congreso de EE UU para aplicar contra Venezuela q… https://t.co/p48SoDAIig
sentiment_value:: pos
confidence:: 1.0
tweet:: RT @b_b_baconator: What he's doing is more easily explained if he fears criminal prosecution. No one in their right mind walks the plank fo…
sentiment_value:: pos
confidence:: 1.0
tweet:: RT @SenWarren: Want to hear something outrageous? @USDOL buried a study on the Trump Admin's plan to make it harder for tipped workers to e…
sentiment_value:: pos
confidence:: 0.6
tweet:: RT @sahilkapur: FBI Director Christopher Wray has told the White House he opposes release of the GOP memo because it cont

tweet:: RT @afbranco: If the memo was going to hurt Trump and the republicans it would have been leaked by now https://t.co/noPLqBdPDn
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @crowdfundernews: Help make it happen for Many Trump Refugees in One Body on #indiegogo https://t.co/ISZr5F6kiv #crowdfunding #comedy #L…
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @PressSec: Americans loved @realDonaldTrump State of the Union - “optimistic,” “hopeful,” “inspiring.” 

https://t.co/TOb3dscOxF
sentiment_value:: pos
confidence:: 1.0
tweet:: RT @kylegriffin1: "Trump suggested to aides and confidants that the memo might give him the justification to fire Rosenstein — something ab…
sentiment_value:: pos
confidence:: 0.6
tweet:: RT @krassenstein: Wow!! Comey basically just called Trump and Nunes "weasels and liars". https://t.co/rPzcmjNvUd
sentiment_value:: neg
confidence:: 0.8
tweet:: RT @GreatCurtuous: Trump said "Americans are dreamers too"
sentiment_value:: neg
confidence:: 0.8
tweet

KeyboardInterrupt: 